In [1]:
import keras
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import display
from keras_tqdm import TQDMNotebookCallback

/home/csci4850/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
data = []
unique = {}
index = 0
y_train = [] # one hots are in here
sizes = []
author_reviews_count = 0
authors = 500
#same_author = True
# officeData has 4905 unique authors
with open('officeDataEncoded.txt') as file:
    for line in file:
        author = line.split('\t')[0].rstrip('\n')
        r = line.split('\t')[1].rstrip('\n')[1:-1].split(',')

        vector = [float(x.strip(',')) for x in r if x != '']
        sizes.append(len(vector))
        if len(unique) == 500:
            break
        
        if author not in unique:
            unique[author] = np.zeros(authors)
            unique[author][index] = 1
            index += 1
            #same_author = False
        if len(vector) == 300:
            data.append(vector)
            y_train.append(unique[author])
            


#data = data

In [3]:
x_tr = []
y_tr = []
x_te = []
y_te = []
#print(data[0])
#print(y_train[0])
cur = y_train[0]
count = 0
for i in range(len(data)):
    if (y_train[i] == cur).all():
        # training set
        if count < 9:
            x_tr.append(data[i])
            y_tr.append(y_train[i])
        # test set
        else:
            x_te.append(data[i])
            y_te.append(y_train[i])

    else:
        #print('HEY')
        cur = y_train[i]
        count = 0
        # training set
        if count < 9:
            x_tr.append(data[i])
            y_tr.append(y_train[i])
        # test set
        else:
            x_te.append(data[i])
            y_te.append(y_train[i])
    count += 1
        

In [4]:
print(len(x_tr))
print(len(y_tr))
print(len(x_te))
print(len(y_te))
#print(y_tr[0:4])
#print(y_te[5])

3566
3566
1729
1729


In [41]:
    print(len(data))
    print(len(y_train[0]))
    print(len(unique))
    reviews = []
    actual_x_training = []
    actual_y_training = []
    actual_x_testing = []
    actual_y_testing = []
    count = 0
    cur = y_train[0]
    for i in range(0, len(data)):
        if y_train[i].all() == cur.all() and count < 5:
            print('1')
            actual_x_training.append(data[i])
            actual_y_training.append(y_train[i])
            count += 1
        elif count >= 5:
            if y_train[i].all() != cur.all():
                actual_x_training.append(data[i])
                actual_y_training.append(y_train[i])
                count = 1
                cur = y_train[i]
            else:
                #print('x')
                actual_x_testing.append(data[i])
                actual_y_testing.append(y_train[i])

#print(actual_x_training)
print(len(actual_x_training))
print(len(actual_y_training))
print(len(actual_x_testing))
print(len(actual_y_testing))

39
5
5
1
1
1
1
1
5
5
34
34


In [5]:
from sklearn.utils import shuffle
x_training,y_training = shuffle(x_tr,y_tr, random_state=0)
#x_test = x_training[:150]
#y_test = y_training[:150]
#x_training = x_training[150:]
#y_training = y_training[150:]
# =====================================================
# =====================================================
x_training = np.array(x_tr)
print(x_training.shape)
# convert the 2d to 3d representation
x_training = x_training.reshape(x_training.shape + (1,))
print(x_training.shape)
output_shape = x_training.shape[1]
input_shape = x_training.shape[2]
y_training = np.array(y_tr)
print('X_training shape: ' + str(x_training.shape))
# =====================================================
# =====================================================

(3566, 300)
(3566, 300, 1)
X_training shape: (3566, 300, 1)


In [24]:
## CURRENT MODEL
model = keras.Sequential()

model.add(keras.layers.Conv1D(700, kernel_size=(3), strides=(1),
                 activation='relu',
                 input_shape=[output_shape,
                              input_shape]))
# not sure if this is needed ...
#model.add(keras.layers.SimpleRNN(200, activation='relu', use_bias=True, 
#                       kernel_initializer='glorot_uniform', recurrent_initializer='orthogonal', 
#                       bias_initializer='zeros', kernel_regularizer=None, recurrent_regularizer=None, 
#                       bias_regularizer=None, activity_regularizer=None, kernel_constraint=None, 
#                       recurrent_constraint=None, bias_constraint=None, dropout=0.0, recurrent_dropout=0.15, 
#                       return_sequences=False, return_state=False, go_backwards=False, stateful=False, unroll=False))

#model.add(keras.layers.MaxPooling2D())
#model.add(keras.layers.Flatten())
model.add(keras.layers.MaxPooling1D())
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(200, activation='relu'))
model.add(keras.layers.Dropout(.15, noise_shape=None, seed=None))
model.add(keras.layers.Dense(200, activation='relu'))
#model.add(keras.layers.Dropout(.25, noise_shape=None, seed=None))
model.add(keras.layers.Dense(y_training.shape[1], activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
    optimizer=keras.optimizers.Adam(),
    metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_3 (Conv1D)            (None, 298, 700)          2800      
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, 149, 700)          0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 104300)            0         
_________________________________________________________________
dense_7 (Dense)              (None, 200)               20860200  
_________________________________________________________________
dropout_3 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 200)               40200     
_________________________________________________________________
dense_9 (Dense)              (None, 500)               100500    
Total para

In [ ]:
batch_size = 50
epochs = 20
history = model.fit(x_training, y_training,
    batch_size=batch_size,
    epochs=epochs,
    verbose=0,
    shuffle=True,
    callbacks=[TQDMNotebookCallback()],
    validation_split = 0.2)

#x_te = np.array(x_te)
#x_te = x_te.reshape(x_te.shape + (1,))
#score = model.evaluate(x_te, np.array([y_te]), verbose=1)
#print('Test loss:', score[0])
#print('Test accuracy:', score[1])

HBox(children=(IntProgress(value=0, description='Training', max=20), HTML(value='')))

HBox(children=(IntProgress(value=0, description='Epoch 0', max=2852), HTML(value='')))

In [22]:
print(x_training.shape)
print(y_training.shape)
print(x_te.shape)
y_te = np.array(y_te)
print(y_te.shape)

(3566, 300, 1)
(3566, 500)
(1729, 300, 1)
(1729, 500)


In [23]:
score = model.evaluate(x_te, y_te, verbose=1)
preds = model.predict_on_batch(x_training)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

1729/1729 [==============================] - 21s 12ms/step
Test loss: 9.67583214057671
Test accuracy: 0.022556390977443608


In [49]:
'''
for y in range(0,len(preds)):
    m = max(preds[y][0:500])
    index = 0
    for i in range(0,500): #preds[0][0:500]:
        if preds[0][i] == m:
            index = i
    print(index)
'''

#m = max(preds[120])
print(len(preds[0]))
index = 0
for i in range(0,5): #preds[0][0:500]:
    print(preds[i])


5
[0.9236263  0.04468539 0.01306041 0.01726184 0.00136603]
[2.1130103e-03 8.6559132e-03 8.6712936e-04 9.8833233e-01 3.1646316e-05]
[0.04776919 0.07349002 0.01289365 0.86393213 0.00191507]
[1.8246796e-02 3.2787237e-02 6.7215497e-03 9.4153255e-01 7.1182550e-04]
[0.05455035 0.08536004 0.02545025 0.8314504  0.00318894]
